### In this notebook

1. Scraping statistics for players in the top-5 european leagues from fbref.com (season 2020/2021)
2. Making statistics relative (x90 minutes or %) so that players with a different number of minutes played could be compared
3. Merging the datasets

### Importing necessary libraries and setting the option to display all columns in a dataframe

In [13]:
import pandas as pd

pd.set_option('display.max_columns', None)

### Loading datasets from fbref.com

In [14]:
standard = pd.read_html('http://fbref.com/en/comps/Big5/2020-2021/stats/players/2020-2021-Big-5-European-Leagues-Stats#stats_standard', header=1)[0]
shooting = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/shooting/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]
passing = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/passing/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]
passing_types = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/passing_types/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]
goal_shot_creation = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/gca/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]
defense = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/defense/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]
possession = pd.read_html('https://fbref.com/en/comps/Big5/2020-2021/possession/players/2020-2021-Big-5-European-Leagues-Stats', header=1)[0]

### Converting into comparable statistic and preparing the datasets to be merged together

In [15]:
##STANDARD DATASET

#Removing useless columns
standard_clean = standard.drop(['Born', 'Matches'], axis=1)

#Removing headers
standard_clean = standard_clean[standard_clean['Rk']!='Rk']

#Converting statistics to proper data type
standard_clean[['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR']] = standard_clean[['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR']].astype('int')
standard_clean[['90s', 'Gls.1', 'Ast.1', 'G+A', 'G-PK.1', 'G+A-PK', 'xG', 'npxG', 'xA', 'npxG+xA', 'xG.1', 'xA.1', 'xG+xA', 'npxG.1', 'npxG+xA.1']] = standard_clean[['90s', 'Gls.1', 'Ast.1', 'G+A', 'G-PK.1', 'G+A-PK', 'xG', 'npxG', 'xA', 'npxG+xA', 'xG.1', 'xA.1', 'xG+xA', 'npxG.1', 'npxG+xA.1']].astype('float')

#Renaming columns for per90 statistics
standard_clean = standard_clean.rename(columns={'Gls':'G', 'Ast':'A', 'Gls.1':'G/90', 'Ast.1':'A/90', 'G+A':'(G+A)/90', 'G-PK.1':'(G-PK)/90', 'G+A-PK':'(G+A-PK)/90', 'xG.1':'xG/90', 'xA.1':'xA/90', 'xG+xA':'(xG+xA)/90', 'npxG.1':'npxG/90', 'npxG+xA.1':'(npxG+xA)/90'})

#Cleaning values in Nation and Competition columns
standard_clean['Nation'] = standard_clean['Nation'].str.split(' ', n = 1, expand = True)[1]
standard_clean['Comp'] = standard_clean['Comp'].str.split(' ', n = 1, expand = True)[1]

#Removing rank column
standard_clean = standard_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
standard_clean['CrdY/90'] = round(standard_clean['CrdY']/standard_clean['90s'], 2)
standard_clean['CrdR/90'] = round(standard_clean['CrdR']/standard_clean['90s'], 2)

std = ['Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'MP', 'Starts', 'Min', '90s', 'CrdY/90', 'CrdR/90', 'G/90', 'A/90', '(G+A)/90', 'PKatt', 'PK', '(G-PK)/90', '(G+A-PK)/90', 'xG/90', 'xA/90', '(xG+xA)/90', 'npxG/90', '(npxG+xA)/90']

standard_clean = standard_clean[std]

print(standard_clean.info())
standard_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        2822 non-null   object 
 1   Nation        2822 non-null   object 
 2   Pos           2822 non-null   object 
 3   Squad         2822 non-null   object 
 4   Comp          2822 non-null   object 
 5   Age           2822 non-null   int64  
 6   MP            2822 non-null   int64  
 7   Starts        2822 non-null   int64  
 8   Min           2822 non-null   int64  
 9   90s           2822 non-null   float64
 10  CrdY/90       2774 non-null   float64
 11  CrdR/90       2775 non-null   float64
 12  G/90          2822 non-null   float64
 13  A/90          2822 non-null   float64
 14  (G+A)/90      2822 non-null   float64
 15  PKatt         2822 non-null   int64  
 16  PK            2822 non-null   int64  
 17  (G-PK)/90     2822 non-null   float64
 18  (G+A-PK)/90   2822 non-null 

,Player,Nation,Pos,Squad,Comp,Age,MP,Starts,Min,90s,CrdY/90,CrdR/90,G/90,A/90,(G+A)/90,PKatt,PK,(G-PK)/90,(G+A-PK)/90,xG/90,xA/90,(xG+xA)/90,npxG/90,(npxG+xA)/90
0,Ismael Aaneba,FRA,"FW,DF",Strasbourg,Ligue 1,21,2,0,11,0.1,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,29,22,20,1777,19.7,0.10,0.00,0.00,0.05,0.05,0,0,0.00,0.05,0.06,0.04,0.10,0.06,0.10
2,Issah Abbas,GHA,"DF,FW",Mainz 05,Bundesliga,21,2,0,18,0.2,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,32,33,33,2889,32.1,0.12,0.03,0.09,0.00,0.09,0,0,0.09,0.09,0.05,0.01,0.06,0.05,0.06
4,Sabit Abdulai,GHA,MF,Getafe,La Liga,21,3,0,60,0.7,1.43,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [16]:
##SHOOTING DATASET

#Removing useless and duplicated columns
shooting_clean = shooting.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Gls', 'PK', 'PKatt', 'xG', 'npxG', 'Matches'], axis=1)

#Removing headers
shooting_clean = shooting_clean[shooting_clean['Rk']!='Rk']

#Converting statistics to proper data type
shooting_clean[['Sh', 'SoT', 'FK']] = shooting_clean[['Sh', 'SoT', 'FK']].astype('int')
shooting_clean[['SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'npxG/Sh', 'G-xG', 'np:G-xG', '90s']] = shooting_clean[['SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'npxG/Sh', 'G-xG', 'np:G-xG', '90s']].astype('float')

#Renaming column
shooting_clean = shooting_clean.rename(columns={'FK':'ShFK'})

#Removing rank column
shooting_clean = shooting_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
shooting_clean['ShFK/Sh%'] = round(shooting_clean['ShFK']/shooting_clean['Sh']*100, 1)
shooting_clean['(G-xG)/90'] = round(shooting_clean['G-xG']/shooting_clean['90s'], 2)
shooting_clean['(np:G-xG)/90'] = round(shooting_clean['np:G-xG']/shooting_clean['90s'], 2)

sht = ['Player', 'Squad', '(G-xG)/90', '(np:G-xG)/90', 'Sh/90', 'SoT/90']
shooting_clean = shooting_clean[sht]

print(shooting_clean.info())
shooting_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Player        2822 non-null   object 
 1   Squad         2822 non-null   object 
 2   (G-xG)/90     2775 non-null   float64
 3   (np:G-xG)/90  2775 non-null   float64
 4   Sh/90         2822 non-null   float64
 5   SoT/90        2822 non-null   float64
dtypes: float64(4), object(2)
memory usage: 154.3+ KB
None


,Player,Squad,(G-xG)/90,(np:G-xG)/90,Sh/90,SoT/90
0,Ismael Aaneba,Strasbourg,0.00,0.00,0.00,0.00
1,Patrick van Aanholt,Crystal Palace,-0.06,-0.06,0.91,0.30
2,Issah Abbas,Mainz 05,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,Reims,0.04,0.04,0.47,0.16
4,Sabit Abdulai,Getafe,0.00,0.00,0.00,0.00


In [17]:
##PASSING DATASET

#Removing useless and duplicated columns
passing_clean = passing.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Matches', 'Ast', 'xA'], axis=1)

#Removing headers
passing_clean = passing_clean[passing_clean['Rk']!='Rk']

#Renaming columns
passing_clean = passing_clean.rename(columns={'Cmp.1':'CmpShort', 'Att.1':'AttShort', 'Cmp%.1':'Cmp%Short', 'Cmp.2':'CmpMed', 'Att.2':'AttMed', 'Cmp%.2':'Cmp%Med', 'Cmp.3':'CmpLong', 'Att.3':'AttLong', 'Cmp%.3':'Cmp%Long', 'xG+xA':'(xG+xA)/90', 'npxG.1':'npxG/90', 'npxG+xA.1':'(npxG+xA)/90'})

#Converting statistics to proper data type
passing_clean[['Cmp', 'Att', 'TotDist', 'PrgDist', 'CmpShort', 'AttShort', 'CmpMed', 'AttMed', 'CmpLong', 'AttLong', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog']] = passing_clean[['Cmp', 'Att', 'TotDist', 'PrgDist', 'CmpShort', 'AttShort', 'CmpMed', 'AttMed', 'CmpLong', 'AttLong', 'KP', '1/3', 'PPA', 'CrsPA', 'Prog']].astype('int')
passing_clean[['Cmp%', 'Cmp%Short', 'Cmp%Med', 'Cmp%Long', 'A-xA', '90s']] = passing_clean[['Cmp%', 'Cmp%Short', 'Cmp%Med', 'Cmp%Long', 'A-xA', '90s']].astype('float')

#Calculating average distance covered by completed passes and proportion of progressive distance 
#over total distance covered by completed passes
passing_clean['AvgDist'] = round(passing_clean['TotDist']/passing_clean['Cmp'], 1)
passing_clean['PrgDist%'] = round(passing_clean['PrgDist']/passing_clean['TotDist']*100,1)

#Removing rank column
passing_clean = passing_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
passing_clean['Att'] = passing_clean['AttShort'] + passing_clean['AttMed'] + passing_clean['AttLong']
passing_clean['PassAtt/90'] = round(passing_clean['Att']/passing_clean['90s'], 2)
passing_clean['Prog/90'] = round(passing_clean['Prog']/passing_clean['90s'], 2)
passing_clean['ShortAtt/90'] = round(passing_clean['AttShort']/passing_clean['90s'], 2)
passing_clean['MedAtt/90'] = round(passing_clean['AttMed']/passing_clean['90s'], 2)
passing_clean['LongAtt/90'] = round(passing_clean['AttLong']/passing_clean['90s'], 2)
passing_clean['(A-xA)/90'] = round(passing_clean['A-xA']/passing_clean['90s'], 2)
passing_clean['KP/90'] = round(passing_clean['KP']/passing_clean['90s'], 2)
passing_clean['(1/3)/90'] = round(passing_clean['1/3']/passing_clean['90s'], 2)
passing_clean['PPA/90'] = round(passing_clean['PPA']/passing_clean['90s'], 2)
passing_clean['CrsPA/90'] = round(passing_clean['CrsPA']/passing_clean['90s'], 2)

pss = ['Player', 'Squad', 'PassAtt/90', 'Prog/90', 'Cmp%', 'ShortAtt/90', 'MedAtt/90', 'LongAtt/90', '(A-xA)/90', 'KP/90', '(1/3)/90', 'PPA/90', 'CrsPA/90']

passing_clean = passing_clean[pss]

print(passing_clean.info())
passing_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Player       2822 non-null   object 
 1   Squad        2822 non-null   object 
 2   PassAtt/90   2809 non-null   float64
 3   Prog/90      2777 non-null   float64
 4   Cmp%         2808 non-null   float64
 5   ShortAtt/90  2797 non-null   float64
 6   MedAtt/90    2792 non-null   float64
 7   LongAtt/90   2782 non-null   float64
 8   (A-xA)/90    2775 non-null   float64
 9   KP/90        2775 non-null   float64
 10  (1/3)/90     2777 non-null   float64
 11  PPA/90       2776 non-null   float64
 12  CrsPA/90     2774 non-null   float64
dtypes: float64(11), object(2)
memory usage: 308.7+ KB
None


,Player,Squad,PassAtt/90,Prog/90,Cmp%,ShortAtt/90,MedAtt/90,LongAtt/90,(A-xA)/90,KP/90,(1/3)/90,PPA/90,CrsPA/90
0,Ismael Aaneba,Strasbourg,20.00,0.00,100.0,0.00,10.00,10.00,0.00,0.00,0.00,0.00,0.0
1,Patrick van Aanholt,Crystal Palace,54.87,5.69,78.9,25.99,21.17,7.72,0.01,0.56,3.45,1.32,0.2
2,Issah Abbas,Mainz 05,75.00,5.00,86.7,35.00,30.00,10.00,0.00,0.00,5.00,0.00,0.0
3,Yunis Abdelhamid,Reims,52.77,2.83,87.8,11.74,29.63,11.40,-0.01,0.12,2.52,0.03,0.0
4,Sabit Abdulai,Getafe,14.29,1.43,58.3,2.86,7.14,4.29,0.00,0.00,0.00,1.43,0.0


In [18]:
##PASSING TYPES DATASET

#Removing useless and duplicated columns. Removing statistics related to corner types as well
passing_types_clean = passing_types.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Matches', 'In', 'Out', 'Str', 'Cmp'], axis=1)

#Removing headers
passing_types_clean = passing_types_clean[passing_types_clean['Rk']!='Rk']

#Converting statistics to proper data type
passing_types_clean[['Att', 'Live', 'Dead', 'FK', 'TB', 'Press', 'Sw', 'Crs', 'CK', 'Ground', 'Low', 'High', 'Left', 'Right', 'Head', 'TI', 'Other', 'Off', 'Out.1', 'Int', 'Blocks']] = passing_types_clean[['Att', 'Live', 'Dead', 'FK', 'TB', 'Press', 'Sw', 'Crs', 'CK', 'Ground', 'Low', 'High', 'Left', 'Right', 'Head', 'TI', 'Other', 'Off', 'Out.1', 'Int', 'Blocks']].astype('int')
passing_types_clean['90s'] = passing_types_clean['90s'].astype('float')

#Dropping the passes attempted and rank columns
passing_types_clean = passing_types_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
passing_types_clean['LiveAtt/90'] = round(passing_types_clean['Live']/passing_types_clean['90s'], 2)
passing_types_clean['DeadAtt/90'] = round(passing_types_clean['Dead']/passing_types_clean['90s'], 2)
passing_types_clean['(PassAtt:FK)/90'] = round(passing_types_clean['FK']/passing_types_clean['90s'], 2)
passing_types_clean['(PassAtt:CK)/90'] = round(passing_types_clean['CK']/passing_types_clean['90s'], 2)
passing_types_clean['(PassAtt:TI)/90'] = round(passing_types_clean['TI']/passing_types_clean['90s'], 2)
passing_types_clean['TB/90'] = round(passing_types_clean['TB']/passing_types_clean['90s'], 2)
passing_types_clean['UndPress/90'] = round(passing_types_clean['Press']/passing_types_clean['90s'], 2)
passing_types_clean['Sw/90'] = round(passing_types_clean['Sw']/passing_types_clean['90s'], 2)
passing_types_clean['Crs/90'] = round(passing_types_clean['Crs']/passing_types_clean['90s'], 2)
passing_types_clean['GroundAtt%'] = round(passing_types_clean['Ground']/passing_types_clean['Att']*100, 1)
passing_types_clean['LowAtt%'] = round(passing_types_clean['Low']/passing_types_clean['Att']*100, 1)
passing_types_clean['HighAtt%'] = round(passing_types_clean['High']/passing_types_clean['Att']*100, 1)
passing_types_clean['LeftAtt%'] = round(passing_types_clean['Left']/passing_types_clean['Att']*100, 1)
passing_types_clean['RightAtt%'] = round(passing_types_clean['Right']/passing_types_clean['Att']*100, 1)
passing_types_clean['HeadAtt%'] = round(passing_types_clean['Head']/passing_types_clean['Att']*100, 1)
passing_types_clean['OtherAtt%'] = round(passing_types_clean['Other']/passing_types_clean['Att']*100, 1)
passing_types_clean['Off%'] = round(passing_types_clean['Off']/passing_types_clean['Att']*100, 1)
passing_types_clean['Out%'] = round(passing_types_clean['Out.1']/passing_types_clean['Att']*100, 1)
passing_types_clean['OppInt%'] = round(passing_types_clean['Int']/passing_types_clean['Att']*100, 1)
passing_types_clean['OppBlk%'] = round(passing_types_clean['Blocks']/passing_types_clean['Att']*100, 1)

pss_type = ['Player', 'Squad', 'LiveAtt/90', 'DeadAtt/90', '(PassAtt:FK)/90', '(PassAtt:CK)/90', '(PassAtt:TI)/90', 'TB/90', 'UndPress/90', 'Sw/90', 'Crs/90', 'GroundAtt%', 'LowAtt%', 'HighAtt%', 'LeftAtt%', 'RightAtt%', 'HeadAtt%', 'OtherAtt%', 'Off%', 'Out%', 'OppInt%', 'OppBlk%']

passing_types_clean = passing_types_clean[pss_type]

print(passing_types_clean.info())
passing_types_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Player           2822 non-null   object 
 1   Squad            2822 non-null   object 
 2   LiveAtt/90       2809 non-null   float64
 3   DeadAtt/90       2779 non-null   float64
 4   (PassAtt:FK)/90  2774 non-null   float64
 5   (PassAtt:CK)/90  2775 non-null   float64
 6   (PassAtt:TI)/90  2777 non-null   float64
 7   TB/90            2774 non-null   float64
 8   UndPress/90      2782 non-null   float64
 9   Sw/90            2777 non-null   float64
 10  Crs/90           2775 non-null   float64
 11  GroundAtt%       2808 non-null   float64
 12  LowAtt%          2808 non-null   float64
 13  HighAtt%         2808 non-null   float64
 14  LeftAtt%         2808 non-null   float64
 15  RightAtt%        2808 non-null   float64
 16  HeadAtt%         2808 non-null   float64
 17  OtherAtt%     

,Player,Squad,LiveAtt/90,DeadAtt/90,(PassAtt:FK)/90,(PassAtt:CK)/90,(PassAtt:TI)/90,TB/90,UndPress/90,Sw/90,Crs/90,GroundAtt%,LowAtt%,HighAtt%,LeftAtt%,RightAtt%,HeadAtt%,OtherAtt%,Off%,Out%,OppInt%,OppBlk%
0,Ismael Aaneba,Strasbourg,20.00,0.00,0.00,0.0,0.00,0.0,0.00,10.00,0.00,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Patrick van Aanholt,Crystal Palace,47.31,9.90,1.83,0.1,7.97,0.1,5.69,1.02,1.73,63.6,19.4,16.9,65.9,17.2,2.2,0.3,1.1,2.0,2.6,3.0
2,Issah Abbas,Mainz 05,60.00,15.00,0.00,0.0,15.00,0.0,0.00,0.00,0.00,66.7,26.7,6.7,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Yunis Abdelhamid,Reims,51.59,1.84,1.84,0.0,0.00,0.0,5.23,1.21,0.00,79.6,7.2,13.2,66.2,27.7,3.3,0.2,0.2,1.5,1.4,0.8
4,Sabit Abdulai,Getafe,17.14,0.00,0.00,0.0,0.00,0.0,10.00,1.43,1.43,66.7,0.0,33.3,8.3,83.3,8.3,0.0,0.0,8.3,8.3,8.3


In [19]:
##GOAL-SHOT CREATION DATASET

#Selecting only most important columns
goal_shot_creation_clean = goal_shot_creation.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Matches'], axis=1)

#Removing headers
goal_shot_creation_clean = goal_shot_creation_clean[goal_shot_creation_clean['Rk']!='Rk']

#Converting statistics to proper data type
goal_shot_creation_clean[['SCA', 'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def', 'GCA', 'PassLive.1', 'PassDead.1','Drib.1', 'Sh.1', 'Fld.1', 'Def.1']] = goal_shot_creation_clean[['SCA', 'PassLive', 'PassDead', 'Drib', 'Sh', 'Fld', 'Def', 'GCA', 'PassLive.1', 'PassDead.1','Drib.1', 'Sh.1', 'Fld.1', 'Def.1']].astype('int')
goal_shot_creation_clean[['SCA90', 'GCA90', '90s']] = goal_shot_creation_clean[['SCA90', 'GCA90', '90s']].astype('float')

#Renaming columns
goal_shot_creation_clean = goal_shot_creation_clean.rename(columns={'PassLive':'SCA:Live', 'PassDead':'SCA:Dead', 'Drib':'SCA:Drib', 'Sh':'SCA:Sh', 'Fld':'SCA:Fld', 'Def':'SCA:Def', 'PassLive.1':'GCA:Live', 'PassDead.1':'GCA:Dead', 'Drib.1':'GCA:Drib', 'Sh.1':'GCA:Sh', 'Fld.1':'GCA:Fld', 'Def.1':'GCA:Def', 'SCA90':'SCA/90', 'GCA90':'GCA/90'})

#Removing rank column
goal_shot_creation_clean = goal_shot_creation_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
goal_shot_creation_clean['(SCA:Live)/90'] = round(goal_shot_creation_clean['SCA:Live']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(SCA:Dead)/90'] = round(goal_shot_creation_clean['SCA:Dead']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(SCA:Drib)/90'] = round(goal_shot_creation_clean['SCA:Drib']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(SCA:Sh)/90'] = round(goal_shot_creation_clean['SCA:Sh']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(SCA:Fld)/90'] = round(goal_shot_creation_clean['SCA:Fld']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(SCA:Def)/90'] = round(goal_shot_creation_clean['SCA:Def']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Live)/90'] = round(goal_shot_creation_clean['GCA:Live']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Dead)/90'] = round(goal_shot_creation_clean['GCA:Dead']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Drib)/90'] = round(goal_shot_creation_clean['GCA:Drib']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Sh)/90'] = round(goal_shot_creation_clean['GCA:Sh']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Fld)/90'] = round(goal_shot_creation_clean['GCA:Fld']/goal_shot_creation_clean['90s'], 2)
goal_shot_creation_clean['(GCA:Def)/90'] = round(goal_shot_creation_clean['GCA:Def']/goal_shot_creation_clean['90s'], 2)

gsc = ['Player', 'Squad', 'SCA/90', '(SCA:Live)/90', '(SCA:Dead)/90', '(SCA:Drib)/90', '(SCA:Sh)/90', '(SCA:Fld)/90', '(SCA:Def)/90', 'GCA/90', '(GCA:Live)/90', '(GCA:Dead)/90', '(GCA:Drib)/90', '(GCA:Sh)/90', '(GCA:Fld)/90', '(GCA:Def)/90']

goal_shot_creation_clean = goal_shot_creation_clean[gsc]

print(goal_shot_creation_clean.info())
goal_shot_creation_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         2822 non-null   object 
 1   Squad          2822 non-null   object 
 2   SCA/90         2822 non-null   float64
 3   (SCA:Live)/90  2779 non-null   float64
 4   (SCA:Dead)/90  2774 non-null   float64
 5   (SCA:Drib)/90  2774 non-null   float64
 6   (SCA:Sh)/90    2774 non-null   float64
 7   (SCA:Fld)/90   2774 non-null   float64
 8   (SCA:Def)/90   2774 non-null   float64
 9   GCA/90         2822 non-null   float64
 10  (GCA:Live)/90  2774 non-null   float64
 11  (GCA:Dead)/90  2774 non-null   float64
 12  (GCA:Drib)/90  2774 non-null   float64
 13  (GCA:Sh)/90    2774 non-null   float64
 14  (GCA:Fld)/90   2774 non-null   float64
 15  (GCA:Def)/90   2774 non-null   float64
dtypes: float64(14), object(2)
memory usage: 374.8+ KB
None


,Player,Squad,SCA/90,(SCA:Live)/90,(SCA:Dead)/90,(SCA:Drib)/90,(SCA:Sh)/90,(SCA:Fld)/90,(SCA:Def)/90,GCA/90,(GCA:Live)/90,(GCA:Dead)/90,(GCA:Drib)/90,(GCA:Sh)/90,(GCA:Fld)/90,(GCA:Def)/90
0,Ismael Aaneba,Strasbourg,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
1,Patrick van Aanholt,Crystal Palace,1.32,1.12,0.15,0.05,0.00,0.0,0.0,0.15,0.1,0.05,0.0,0.0,0.0,0.0
2,Issah Abbas,Mainz 05,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
3,Yunis Abdelhamid,Reims,0.34,0.31,0.00,0.00,0.03,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0
4,Sabit Abdulai,Getafe,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0


In [20]:
##DEFENSE DATASET

#Removing useless and duplicated columns. Removing errors leading to an opponent's shot as well
defense_clean = defense.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Err', 'Matches'], axis=1)

#Removing headers
defense_clean = defense_clean[defense_clean['Rk']!='Rk']

#Converting statistics to proper data type
defense_clean[['Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl.1', 'Att', 'Past', 'Press','Succ', 'Def 3rd.1', 'Mid 3rd.1', 'Att 3rd.1', 'Blocks', 'Sh', 'ShSv', 'Pass', 'Int', 'Tkl+Int', 'Clr']] = defense_clean[['Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl.1', 'Att', 'Past', 'Press','Succ', 'Def 3rd.1', 'Mid 3rd.1', 'Att 3rd.1', 'Blocks', 'Sh', 'ShSv', 'Pass', 'Int', 'Tkl+Int', 'Clr']].astype('int')
defense_clean[['Tkl%', '%', '90s']] = defense_clean[['Tkl%', '%', '90s']].astype('float')

#Removing rank column
defense_clean = defense_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
defense_clean['Tkl/90'] = round(defense_clean['Tkl']/defense_clean['90s'], 2)
defense_clean['TklW/90'] = round(defense_clean['TklW']/defense_clean['90s'], 2)
defense_clean['(Tkl:Def)/90'] = round(defense_clean['Def 3rd']/defense_clean['90s'], 2)
defense_clean['(Tkl:Mid)/90'] = round(defense_clean['Mid 3rd']/defense_clean['90s'], 2)
defense_clean['(Tkl:Att)/90'] = round(defense_clean['Att 3rd']/defense_clean['90s'], 2)
defense_clean['DriTkl/90'] = round(defense_clean['Att']/defense_clean['90s'], 2)
defense_clean['DriTklW/90'] = round(defense_clean['Tkl.1']/defense_clean['90s'], 2)
defense_clean['Press/90'] = round(defense_clean['Press']/defense_clean['90s'], 2)
defense_clean['SuccPress/90'] = round(defense_clean['Succ']/defense_clean['90s'], 2)
defense_clean['(Press:Def)/90'] = round(defense_clean['Def 3rd.1']/defense_clean['90s'], 2)
defense_clean['(Press:Mid)/90'] = round(defense_clean['Mid 3rd.1']/defense_clean['90s'], 2)
defense_clean['(Press:Att)/90'] = round(defense_clean['Att 3rd.1']/defense_clean['90s'], 2)
defense_clean['Blk/90'] = round(defense_clean['Blocks']/defense_clean['90s'], 2)
defense_clean['Blk:Sh/90'] = round(defense_clean['Sh']/defense_clean['90s'], 2)
defense_clean['Blk:ShoT/90'] = round(defense_clean['ShSv']/defense_clean['90s'], 2)
defense_clean['Int/90'] = round(defense_clean['Int']/defense_clean['90s'], 2)
defense_clean['(Tkl+Int)/90'] = round(defense_clean['Tkl+Int']/defense_clean['90s'], 2)
defense_clean['Clr/90'] = round(defense_clean['Clr']/defense_clean['90s'], 2)

de = ['Player', 'Squad', 'Tkl/90', 'TklW/90', '(Tkl:Def)/90', '(Tkl:Mid)/90', '(Tkl:Att)/90', 'DriTkl/90', 'DriTklW/90', 'Press/90', 'SuccPress/90', '(Press:Def)/90', '(Press:Mid)/90', '(Press:Att)/90', 'Blk/90', 'Blk:Sh/90', 'Blk:ShoT/90', 'Int/90', '(Tkl+Int)/90', 'Clr/90']

defense_clean = defense_clean[de]

print(defense_clean.info())
defense_clean.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Player          2822 non-null   object 
 1   Squad           2822 non-null   object 
 2   Tkl/90          2780 non-null   float64
 3   TklW/90         2776 non-null   float64
 4   (Tkl:Def)/90    2777 non-null   float64
 5   (Tkl:Mid)/90    2777 non-null   float64
 6   (Tkl:Att)/90    2775 non-null   float64
 7   DriTkl/90       2780 non-null   float64
 8   DriTklW/90      2776 non-null   float64
 9   Press/90        2806 non-null   float64
 10  SuccPress/90    2791 non-null   float64
 11  (Press:Def)/90  2792 non-null   float64
 12  (Press:Mid)/90  2792 non-null   float64
 13  (Press:Att)/90  2785 non-null   float64
 14  Blk/90          2777 non-null   float64
 15  Blk:Sh/90       2774 non-null   float64
 16  Blk:ShoT/90     2774 non-null   float64
 17  Int/90          2776 non-null   f

,Player,Squad,Tkl/90,TklW/90,(Tkl:Def)/90,(Tkl:Mid)/90,(Tkl:Att)/90,DriTkl/90,DriTklW/90,Press/90,SuccPress/90,(Press:Def)/90,(Press:Mid)/90,(Press:Att)/90,Blk/90,Blk:Sh/90,Blk:ShoT/90,Int/90,(Tkl+Int)/90,Clr/90
0,Ismael Aaneba,Strasbourg,0.00,0.00,0.00,0.00,0.00,10.00,0.00,20.00,0.00,10.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00
1,Patrick van Aanholt,Crystal Palace,1.68,0.96,1.07,0.56,0.05,2.13,0.76,10.46,3.50,6.60,2.84,1.02,1.57,0.36,0.05,1.73,3.40,2.03
2,Issah Abbas,Mainz 05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.00,0.00,0.00,5.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Yunis Abdelhamid,Reims,2.21,1.40,1.59,0.62,0.00,1.15,0.47,11.81,4.55,7.85,3.43,0.53,1.59,0.62,0.06,1.74,3.96,5.30
4,Sabit Abdulai,Getafe,1.43,0.00,0.00,1.43,0.00,1.43,0.00,22.86,5.71,7.14,11.43,4.29,0.00,0.00,0.00,1.43,2.86,1.43


In [21]:
##POSSESSION DATASET

#Removing useless and duplicated columns. Removing nutmegs and # of players dribbled as well
possession_clean = possession.drop(['Nation', 'Pos', 'Comp', 'Age', 'Born', 'Matches', 'Megs', '#Pl'], axis=1)

#Removing headers
possession_clean = possession_clean[possession_clean['Rk']!='Rk']

#Converting statistics to proper data type
possession_clean[['Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Live', 'Succ', 'Att', 'Carries', 'TotDist', 'PrgDist', 'Prog', '1/3', 'CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Prog.1']] = possession_clean[['Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Live', 'Succ', 'Att', 'Carries', 'TotDist', 'PrgDist', 'Prog', '1/3', 'CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Prog.1']].astype('int')
possession_clean[['Succ%', 'Rec%', '90s']] = possession_clean[['Succ%', 'Rec%', '90s']].astype('float')

#Removing rank column
possession_clean = possession_clean.drop(['Rk'], axis=1)

#Converting into comparable statistics
possession_clean['Tch'] = possession_clean['Def 3rd'] + possession_clean['Mid 3rd'] + possession_clean['Att 3rd']
possession_clean['Tch/90'] = round(possession_clean['Tch']/possession_clean['90s'], 2)
possession_clean['TchDef:Pen%'] = round(possession_clean['Att Pen']/possession_clean['Att 3rd']*100, 1)
possession_clean['TchAtt:Pen%'] = round(possession_clean['Att Pen']/possession_clean['Att 3rd']*100, 1)
possession_clean['Tch:Def%'] = round(possession_clean['Def 3rd']/possession_clean['Tch']*100, 1)
possession_clean['Tch:Mid%'] = round(possession_clean['Mid 3rd']/possession_clean['Tch']*100, 1)
possession_clean['Tch:Att%'] = round(possession_clean['Att 3rd']/possession_clean['Tch']*100, 1)
possession_clean['Tch:Live%'] = round(possession_clean['Live']/possession_clean['Tch']*100, 1)
possession_clean['Dri/90'] = round(possession_clean['Att']/possession_clean['90s'], 2)
possession_clean['DriW%'] = round(possession_clean['Succ']/possession_clean['Att']*100, 1)
possession_clean['Carries/90'] = round(possession_clean['Carries']/possession_clean['90s'], 2)
possession_clean['CarrAvgDist'] = round(possession_clean['TotDist']/possession_clean['Carries'], 2)
possession_clean['PCarrDist%'] = round(possession_clean['PrgDist']/possession_clean['TotDist']*100, 1)
possession_clean['PCarries/90'] = round(possession_clean['Prog']/possession_clean['90s'], 2)
possession_clean['(1/3 Carr)/90'] = round(possession_clean['1/3']/possession_clean['90s'], 2)
possession_clean['CPA/90'] = round(possession_clean['CPA']/possession_clean['90s'], 2)
possession_clean['Mis/90'] = round(possession_clean['Mis']/possession_clean['90s'], 2)
possession_clean['Dis/90'] = round(possession_clean['Dis']/possession_clean['90s'], 2)
possession_clean['Targ/90'] = round(possession_clean['Targ']/possession_clean['90s'], 2)
possession_clean['Rec:Prog%'] = round(possession_clean['Prog.1']/possession_clean['Rec']*100, 1)

poss = ['Player', 'Squad', 'Tch/90', 'TchDef:Pen%', 'Tch:Def%', 'Tch:Mid%', 'Tch:Att%', 'TchAtt:Pen%', 'Tch:Live%', 'Dri/90', 'DriW%', 'Carries/90', 'CarrAvgDist', 'PCarrDist%', 'PCarries/90', '(1/3 Carr)/90', 'CPA/90', 'Mis/90', 'Dis/90', 'Targ/90', 'Rec%', 'Rec:Prog%']
possession_clean = possession_clean[poss]

print(possession_clean.info())
possession_clean.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 0 to 2933
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Player         2822 non-null   object 
 1   Squad          2822 non-null   object 
 2   Tch/90         2812 non-null   float64
 3   TchDef:Pen%    2580 non-null   float64
 4   Tch:Def%       2812 non-null   float64
 5   Tch:Mid%       2812 non-null   float64
 6   Tch:Att%       2812 non-null   float64
 7   TchAtt:Pen%    2580 non-null   float64
 8   Tch:Live%      2812 non-null   float64
 9   Dri/90         2778 non-null   float64
 10  DriW%          2370 non-null   float64
 11  Carries/90     2804 non-null   float64
 12  CarrAvgDist    2800 non-null   float64
 13  PCarrDist%     2788 non-null   float64
 14  PCarries/90    2781 non-null   float64
 15  (1/3 Carr)/90  2775 non-null   float64
 16  CPA/90         2777 non-null   float64
 17  Mis/90         2780 non-null   float64
 18  Dis/90  

,Player,Squad,Tch/90,TchDef:Pen%,Tch:Def%,Tch:Mid%,Tch:Att%,TchAtt:Pen%,Tch:Live%,Dri/90,DriW%,Carries/90,CarrAvgDist,PCarrDist%,PCarries/90,(1/3 Carr)/90,CPA/90,Mis/90,Dis/90,Targ/90,Rec%,Rec:Prog%
0,Ismael Aaneba,Strasbourg,50.00,NaN,80.0,20.0,0.0,NaN,80.0,0.00,NaN,20.00,5.00,40.0,0.00,0.00,0.00,0.00,0.00,30.00,33.3,0.0
1,Patrick van Aanholt,Crystal Palace,70.66,11.0,29.1,46.2,24.7,11.0,80.9,1.07,42.9,37.21,4.80,54.1,5.79,1.52,0.46,0.66,0.81,42.39,92.8,5.9
2,Issah Abbas,Mainz 05,95.00,12.5,26.3,31.6,42.1,12.5,78.9,10.00,50.0,55.00,4.27,85.1,10.00,5.00,0.00,0.00,5.00,60.00,91.7,18.2
3,Yunis Abdelhamid,Reims,69.47,51.2,53.9,44.2,1.8,51.2,91.0,0.62,60.0,41.71,5.65,55.4,1.81,0.25,0.03,0.25,0.47,43.83,97.0,0.5
4,Sabit Abdulai,Getafe,27.14,0.0,10.5,57.9,31.6,0.0,89.5,4.29,33.3,15.71,4.18,47.8,1.43,1.43,0.00,0.00,0.00,14.29,80.0,12.5


### Merging datasets and exporting the main df

In [22]:
df = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(standard_clean, shooting_clean,on=['Player', 'Squad']), passing_clean, on=['Player', 'Squad']), passing_types_clean, on=['Player', 'Squad']), goal_shot_creation_clean, on=['Player', 'Squad']), defense_clean, on=['Player', 'Squad']), possession_clean, on=['Player', 'Squad'])

df.to_csv('df.csv', index=False)

print(df.shape)
df.head()

(2822, 111)


,Player,Nation,Pos,Squad,Comp,Age,MP,Starts,Min,90s,CrdY/90,CrdR/90,G/90,A/90,(G+A)/90,PKatt,PK,(G-PK)/90,(G+A-PK)/90,xG/90,xA/90,(xG+xA)/90,npxG/90,(npxG+xA)/90,(G-xG)/90,(np:G-xG)/90,Sh/90,SoT/90,PassAtt/90,Prog/90,Cmp%,ShortAtt/90,MedAtt/90,LongAtt/90,(A-xA)/90,KP/90,(1/3)/90,PPA/90,CrsPA/90,LiveAtt/90,DeadAtt/90,(PassAtt:FK)/90,(PassAtt:CK)/90,(PassAtt:TI)/90,TB/90,UndPress/90,Sw/90,Crs/90,GroundAtt%,LowAtt%,HighAtt%,LeftAtt%,RightAtt%,HeadAtt%,OtherAtt%,Off%,Out%,OppInt%,OppBlk%,SCA/90,(SCA:Live)/90,(SCA:Dead)/90,(SCA:Drib)/90,(SCA:Sh)/90,(SCA:Fld)/90,(SCA:Def)/90,GCA/90,(GCA:Live)/90,(GCA:Dead)/90,(GCA:Drib)/90,(GCA:Sh)/90,(GCA:Fld)/90,(GCA:Def)/90,Tkl/90,TklW/90,(Tkl:Def)/90,(Tkl:Mid)/90,(Tkl:Att)/90,DriTkl/90,DriTklW/90,Press/90,SuccPress/90,(Press:Def)/90,(Press:Mid)/90,(Press:Att)/90,Blk/90,Blk:Sh/90,Blk:ShoT/90,Int/90,(Tkl+Int)/90,Clr/90,Tch/90,TchDef:Pen%,Tch:Def%,Tch:Mid%,Tch:Att%,TchAtt:Pen%,Tch:Live%,Dri/90,DriW%,Carries/90,CarrAvgDist,PCarrDist%,PCarries/90,(1/3 Carr)/90,CPA/90,Mis/90,Dis/90,Targ/90,Rec%,Rec:Prog%
0,Ismael Aaneba,FRA,"FW,DF",Strasbourg,Ligue 1,21,2,0,11,0.1,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,0.00,100.0,0.00,10.00,10.00,0.00,0.00,0.00,0.00,0.0,20.00,0.00,0.00,0.0,0.00,0.0,0.00,10.00,0.00,100.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,10.00,0.00,20.00,0.00,10.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,20.00,50.00,NaN,80.0,20.0,0.0,NaN,80.0,0.00,NaN,20.00,5.00,40.0,0.00,0.00,0.00,0.00,0.00,30.00,33.3,0.0
1,Patrick van Aanholt,NED,DF,Crystal Palace,Premier League,29,22,20,1777,19.7,0.10,0.00,0.00,0.05,0.05,0,0,0.00,0.05,0.06,0.04,0.10,0.06,0.10,-0.06,-0.06,0.91,0.30,54.87,5.69,78.9,25.99,21.17,7.72,0.01,0.56,3.45,1.32,0.2,47.31,9.90,1.83,0.1,7.97,0.1,5.69,1.02,1.73,63.6,19.4,16.9,65.9,17.2,2.2,0.3,1.1,2.0,2.6,3.0,1.32,1.12,0.15,0.05,0.00,0.0,0.0,0.15,0.1,0.05,0.0,0.0,0.0,0.0,1.68,0.96,1.07,0.56,0.05,2.13,0.76,10.46,3.50,6.60,2.84,1.02,1.57,0.36,0.05,1.73,3.40,2.03,70.66,11.0,29.1,46.2,24.7,11.0,80.9,1.07,42.9,37.21,4.80,54.1,5.79,1.52,0.46,0.66,0.81,42.39,92.8,5.9
2,Issah Abbas,GHA,"DF,FW",Mainz 05,Bundesliga,21,2,0,18,0.2,0.00,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,75.00,5.00,86.7,35.00,30.00,10.00,0.00,0.00,5.00,0.00,0.0,60.00,15.00,0.00,0.0,15.00,0.0,0.00,0.00,0.00,66.7,26.7,6.7,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,15.00,0.00,0.00,5.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,95.00,12.5,26.3,31.6,42.1,12.5,78.9,10.00,50.0,55.00,4.27,85.1,10.00,5.00,0.00,0.00,5.00,60.00,91.7,18.2
3,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,32,33,33,2889,32.1,0.12,0.03,0.09,0.00,0.09,0,0,0.09,0.09,0.05,0.01,0.06,0.05,0.06,0.04,0.04,0.47,0.16,52.77,2.83,87.8,11.74,29.63,11.40,-0.01,0.12,2.52,0.03,0.0,51.59,1.84,1.84,0.0,0.00,0.0,5.23,1.21,0.00,79.6,7.2,13.2,66.2,27.7,3.3,0.2,0.2,1.5,1.4,0.8,0.34,0.31,0.00,0.00,0.03,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,2.21,1.40,1.59,0.62,0.00,1.15,0.47,11.81,4.55,7.85,3.43,0.53,1.59,0.62,0.06,1.74,3.96,5.30,69.47,51.2,53.9,44.2,1.8,51.2,91.0,0.62,60.0,41.71,5.65,55.4,1.81,0.25,0.03,0.25,0.47,43.83,97.0,0.5
4,Sabit Abdulai,GHA,MF,Getafe,La Liga,21,3,0,60,0.7,1.43,0.00,0.00,0.00,0.00,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.29,1.43,58.3,2.86,7.14,4.29,0.00,0.00,0.00,1.43,0.0,17.14,0.00,0.00,0.0,0.00,0.0,10.00,1.43,1.43,66.7,0.0,33.3,8.3,83.3,8.3,0.0,0.0,8.3,8.3,8.3,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,1.43,0.00,0.00,1.43,0.00,1.43,0.00,22.86,5.71,7.14,11.43,4.29,0.00,0.00,0.00,1.43,2.86,1.43,27.14,0.0,10.5,57.9,31.6,0.0,89.5,4.29,33.3,15.71,4.18,47.8,1.43,1.43,0.00,0.00,0.00,14.29,80.0,12.5
